In [6]:
import numpy as np
import time
import pickle
import utils as Utils
import pandas as pd
from sklearn import svm
import seaborn as sns
import matplotlib.pyplot as plt
# import modules.model as Model
from matplotlib.backends.backend_pdf import PdfPages
import sklearn.metrics as metrics

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, plot_roc_curve
from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score,roc_auc_score,classification_report,roc_curve,auc, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import plotly.graph_objects as go
from imblearn.over_sampling import SMOTE 
# import modules.processor as Processor
# import modules.utils as Utils
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
import xgboost as xgb
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from tqdm import tqdm
import processor as Processor

In [13]:
# !pip3 install xgboost

In [2]:
train_trb = pd.read_csv("./data_final/train_trb.csv")
test_trb = pd.read_csv("./data_final/test_trb.csv")

In [3]:
nm = RandomUnderSampler(random_state=42)
X_res, y_res = nm.fit_resample(train_trb[["CDR3b", "epitope"]], train_trb[["binder"]])

In [4]:
pX_res = Processor.data_representation_blosum62(X_res)
py_res = y_res.copy()

pX_test = Processor.data_representation_blosum62(test_trb)
pX_test = test_trb[["binder"]]

In [5]:
print(pX_res.shape, py_res.shape)
print(pX_test.shape, pX_test.shape)

(83874, 600) (83874, 1)
(1385101, 1) (1385101, 1)


In [7]:
lst_models = [
    ('Logistic Regression - [solver: lbfgs]', LogisticRegression(solver='lbfgs')),
    ('Logistic Regression - [solver: liblinear]', LogisticRegression(solver='liblinear')),
    ('Logistic Regression - [solver: newton-cg]', LogisticRegression(solver='newton-cg')),
    ('KNN - [n_neighbors: 2]', KNeighborsClassifier(n_neighbors=2)),
    ('KNN - [n_neighbors: 3]', KNeighborsClassifier(n_neighbors=3)),
#     ('SVC - [kernel: linear]', SVC(kernel='linear', random_state=42)),
#     ('SVC - [kernel: poly]', SVC(kernel='poly', random_state=42)),
#     ('SVC - [kernel: rbf]', SVC(kernel='rbf', random_state=42)),
#     ('SVC - [kernel: sigmoid]', SVC(kernel='sigmoid', random_state=42)),
    ('Bernoulli', BernoulliNB()),
    ('Decision Tree', DecisionTreeClassifier(random_state=42)),
    ('Random Forest', RandomForestClassifier(random_state=42, n_jobs=-1)),]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
def train(lst_models, X, y, cv):
    res_table = []
    for mdl_name, model in tqdm(lst_models):
        tic = time.time()
        cv_res = cross_validate(model, X, y, cv=cv, return_train_score=True, scoring=['accuracy', 'roc_auc'], n_jobs=-1)
        res_table.append([mdl_name, 
                          cv_res['train_accuracy'].mean(), 
                          cv_res['test_accuracy'].mean(), 
                          np.abs(cv_res['train_accuracy'].mean() - cv_res['test_accuracy'].mean()),
                          cv_res['train_accuracy'].std(),
                          cv_res['test_accuracy'].std(),
                          cv_res['train_roc_auc'].mean(),
                          cv_res['test_roc_auc'].mean(),
                          np.abs(cv_res['train_roc_auc'].mean() - cv_res['test_roc_auc'].mean()),
                          cv_res['train_roc_auc'].std(),
                          cv_res['test_roc_auc'].std(),
                          cv_res['fit_time'].mean()
        ])
        toc = time.time()
        print('\tModel {} has been trained in {:,.2f} seconds'.format(mdl_name, (toc - tic)))
    
    res_table = pd.DataFrame(res_table, columns=['model', 'train_acc', 'test_acc', 'diff_acc',
                                                 'train_acc_std', 'test_acc_std', 'train_roc_auc', 'test_roc_auc',
                                                 'diff_roc_auc', 'train_roc_auc_std', 'test_roc_auc_std', 'fit_time'])
    res_table.sort_values(by=['test_acc', 'test_roc_auc'], ascending=False, inplace=True)

    return res_table.reset_index(drop=True)

In [9]:
original_models = train(lst_models, pX_res, np.ravel(py_res), cv)
original_models

 12%|█▎        | 1/8 [00:23<02:42, 23.28s/it]

	Model Logistic Regression - [solver: lbfgs] has been trained in 23.28 seconds


 25%|██▌       | 2/8 [04:20<14:53, 148.87s/it]

	Model Logistic Regression - [solver: liblinear] has been trained in 236.79 seconds


 38%|███▊      | 3/8 [14:06<29:02, 348.55s/it]

...

 88%|████████▊ | 7/8 [27:22<02:54, 174.10s/it]

	Model Decision Tree has been trained in 19.70 seconds


100%|██████████| 8/8 [27:34<00:00, 206.85s/it]

	Model Random Forest has been trained in 12.59 seconds


,model,train_acc,test_acc,diff_acc,train_acc_std,test_acc_std,train_roc_auc,test_roc_auc,diff_roc_auc,train_roc_auc_std,test_roc_auc_std,fit_time
0,Random Forest,0.999994,0.912595,0.087399,0.000007,0.001636,1.000000,0.977712,0.022288,0.000000,0.000499,10.283544
1,Logistic Regression - [solver: newton-cg],0.913775,0.909924,0.003851,0.000310,0.000986,0.977165,0.974782,0.002383,0.000144,0.000565,463.505757
2,Logistic Regression - [solver: liblinear],0.913778,0.909912,0.003866,0.000319,0.000978,0.977164,0.974781,0.002383,0.000144,0.000565,200.388664
3,Logistic Regression - [solver: lbfgs],0.910804,0.908029,0.002775,0.000484,0.000341,0.975064,0.973549,0.001515,0.000188,0.000840,5.575970
4,KNN - [n_neighbors: 3],0.941063,0.893400,0.047664,0.000828,0.000925,0.989432,0.943072,0.046360,0.000146,0.001855,0.531955
5,Decision Tree,1.000000,0.886556,0.113444,0.000000,0.003429,1.000000,0.886556,0.113444,0.000000,0.003429,18.127080
6,KNN - [n_neighbors: 2],0.940837,0.881477,0.059360,0.000352,0.001384,0.993262,0.925701,0.067561,0.000068,0.001675,0.514485
7,Bernoulli,0.845044,0.844803,0.000241,0.000569,0.003019,0.906436,0.905731,0.000705,0.000630,0.002698,0.913533


In [13]:
lst_tunning_models = [
     ('Random Forest', RandomForestClassifier(random_state=42, n_jobs=-1), {
        'bootstrap': [True, False],
        'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
        'max_features': ["auto", "sqrt", "log2", 5, 10, 15, 20],
        'min_samples_leaf': [1, 2, 4],
        'n_estimators': [200, 400, 600, 800, 1000]})]

def trainTunningModel(lst_models, X, y, cv):
    models_final = []
    for model_name, model, params in tqdm(lst_models):
        tic     = time.time()
        search = GridSearchCV(estimator=model, param_grid=params, cv=cv, scoring='accuracy', n_jobs=-1)
        search.fit(X, y)
        model_tunned = model.set_params(**search.best_params_)
        models_final.append((model_name, model_tunned))
        toc = time.time()
        print('Model {} has been tunned in {:,.2f} seconds'.format(model_name, (toc - tic)))

    return models_final

In [15]:
# lst_tunning_models = trainTunningModel(lst_tunning_models, pX_res, np.ravel(py_res), cv)
# lst_tunning_models

In [ ]:
# lst_tunning_models